1. 네이버 영화리뷰 감정 분석 코퍼스에 SentencePiece를 적용시킨 모델 학습하기
2. 학습된 모델로 sp_tokenize() 메소드 구현하기
3. 구현된 토크나이저를 적용하여 네이버 영화리뷰 감정 분석 모델을 재학습하기  

시간상 여기까지만 완료했습니다,,

4. KoNLPy 형태소 분석기를 사용한 모델과 성능 비교하기
5. SentencePiece 모델의 model_type, vocab_size 등을 변경해 가면서 성능 개선 여부 확인하기

# 1. 라이브러리 및 데이터 로드

In [3]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 42.1 MB/s eta 0:00:00


In [4]:
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import konlpy

print(torch.__version__)
print(np.__version__)
print(pd.__version__)
print(matplotlib.__version__)
print(konlpy.__version__)

2.8.0+cu126
2.0.2
2.2.2
3.10.0
0.6.0


In [5]:
# 네이버 영화리뷰 감정분석 데이터
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

--2025-11-13 07:23:39--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.03s   

2025-11-13 07:23:40 (437 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]

--2025-11-13 07:23:40--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [application/octet-

In [6]:
train = pd.read_table("/content/ratings_train.txt")
test = pd.read_table("/content/ratings_test.txt")

print(train.info())
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
None


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 2. 전처리

In [8]:
import re

# 불용어 리스트
STOPWORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 맞춤법 변형 사전
SPELLING_DICT = {
    '굳': ['궅', '굳', '굿'],
    '미쳤': ['미첫', '미쳣', '미첬', '미쳤', 'ㅁㅊ'],
    '괜찮': ['괜찮', '괜춘', '괜찬', 'ㄱㅊ', '갠찬', '갠찮', '괸찬', '괸찮'],
    '봤': ['봣'],
    '겠': ['겟']
}

# 텍스트 컬럼명
TEXT_COL = "document"

def preprocess_text(text):
    """
    텍스트 전처리 함수
    """
    if pd.isna(text):
        return ""

    text = str(text)

    # 1. 반복되는 문장 부호 제거 (2개 이상 → 1개)
    text = re.sub(r'([.!?…;])\1+', r'\1', text)

    # 2. 맞춤법 변형 통일
    for correct, variations in SPELLING_DICT.items():
        for variant in variations:
            text = text.replace(variant, correct)

    # 3. 반복 문자 제거 (3번 이상 반복 → 2번)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # 4. 자음/모음 단독 제거 (완성형 한글 필터링 전에 먼저 제거)
    # 한글 자음: ㄱ-ㅎ, 한글 모음: ㅏ-ㅣ
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', text)

    # 5. 영어/숫자/한글/문장부호만 남기고 모두 삭제
    # 이모지, 이모티콘, 특수문자 자동 제거
    text = re.sub(r'[^가-힣a-zA-Z0-9\s.!?,]', ' ', text)

    # 6. 문장부호 앞뒤로 공백 추가
    text = re.sub(r'([.!?,])', r' \1 ', text)

    # 7. 영어 소문자 변환
    text = text.lower()

    # 9. 불용어 제거 (조사가 붙은 경우도 처리)
    words = text.split()

    # 단어 끝에 불용어가 붙어있으면 제거
    filtered_words = []
    for word in words:
        # 단어 전체가 불용어인 경우
        if word in STOPWORDS:
            continue
        # 단어 끝에서 불용어 제거 (가장 긴 것부터 체크)
        # 예: "학교에서" -> "학교", "영화는" -> "영화"
        found = False
        for stopword in sorted(STOPWORDS, key=len, reverse=True):
            if len(word) > len(stopword) and word.endswith(stopword):
                cleaned = word[:-len(stopword)]
                # 남은 부분이 2글자 이상일 때만 제거
                # 굳이 -> 굳, 같이 -> 같  이렇게 바뀌어서 길이 제한 추가했습니다.
                # "굳이" -> "굳" (X), "영화는" -> "영화" (O)
                if len(cleaned) >= 2:
                    filtered_words.append(cleaned)
                    found = True
                    break
        if not found:
            filtered_words.append(word)

    text = ' '.join(filtered_words)

    # 10. 연속 공백을 하나의 공백으로 교체
    text = re.sub(r'\s+', ' ', text)

    # 11. 앞뒤 공백 제거
    text = text.strip()

    return text


def preprocess_dataframe(df, text_col=TEXT_COL):
    """
    데이터프레임 전처리 함수
    """
    print(f"전처리 전 데이터 크기: {len(df)}")

    # 1. 결측치 제거
    df = df.dropna(subset=[text_col])
    print(f"결측치 제거 후: {len(df)}")

    # 2. 텍스트 전처리 적용
    df[text_col] = df[text_col].apply(preprocess_text)

    # 3. 전처리 후 빈 문자열 제거
    df = df[df[text_col].str.strip() != '']
    print(f"빈 문자열 제거 후: {len(df)}")

    # 4. 중복 행 제거
    df = df.drop_duplicates(subset=[text_col])
    print(f"중복 제거 후: {len(df)}")

    # 인덱스 재설정
    df = df.reset_index(drop=True)

    return df


# 샘플 텍스트로 테스트
sample_texts = [
    "어제 본 영화 진짜 재밌었음!!! 또 보고 싶어 😂",
    "나는 오늘 아침에 학교에 갔다. 근데 너무 졸렸음ㅋㅋㅋㅋ",
    "밥은 먹었니?? 아직이야... 점심에 같이 먹자!!!",
    "메캅 형태소 분석은 한국어 처리에서 많이 사용돼 👍",
    "파이썬으로 토큰 빈도와 품사 분포를 시각화해 보자!!!",
    "요즘 코사인 유사도 기반 벡터 검색으로 RG 구축을 많이 해!!",
    "에이전트는 외부 도구를 호출해 작업을 자동화할 수 있어. 굳!",
    "이 영화 진짜 미쳣다!!! 너무 재밌음ㅋㅋㅋㅋ",
    "배우 연기 굳이 훌륭했음, 스토리는 봣지만...",
    "이건 ㄱㅊ 영화네, 굿굿!"
]

print("=" * 80)
print("샘플 텍스트 전처리 결과")
print("=" * 80)
for i, text in enumerate(sample_texts, 1):
    processed = preprocess_text(text)
    print(f"\n[{i}] 원본: {text}")
    print(f"    결과: {processed}")

print("\n" + "=" * 80)
print("실제 데이터 적용 예시")
print("=" * 80)

train_processed = preprocess_dataframe(train.copy())
test_processed = preprocess_dataframe(test.copy())

print("\n전처리 완료!")
print(f"Train 데이터: {len(train_processed)}개")
print(f"Test 데이터: {len(test_processed)}개")

print("\n전처리 결과 샘플:")
print(train_processed.head(10))

샘플 텍스트 전처리 결과

[1] 원본: 어제 본 영화 진짜 재밌었음!!! 또 보고 싶어 😂
    결과: 어제 본 영화 진짜 재밌었음 ! 또 보고 싶어

[2] 원본: 나는 오늘 아침에 학교에 갔다. 근데 너무 졸렸음ㅋㅋㅋㅋ
    결과: 나는 오늘 아침 학교 갔다 . 근데 너무 졸렸음

[3] 원본: 밥은 먹었니?? 아직이야... 점심에 같이 먹자!!!
    결과: 밥은 먹었니 ? 아직이야 . 점심 같이 먹자 !

[4] 원본: 메캅 형태소 분석은 한국어 처리에서 많이 사용돼 👍
    결과: 메캅 형태소 분석 한국어 처리에서 많이 사용돼

[5] 원본: 파이썬으로 토큰 빈도와 품사 분포를 시각화해 보자!!!
    결과: 파이썬 토큰 빈도 품사 분포 시각화해 보자 !

[6] 원본: 요즘 코사인 유사도 기반 벡터 검색으로 RG 구축을 많이 해!!
    결과: 요즘 코사인 유사 기반 벡터 검색 rg 구축을 많이 해 !

[7] 원본: 에이전트는 외부 도구를 호출해 작업을 자동화할 수 있어. 굳!
    결과: 에이전트 외부 도구 호출해 작업을 자동화할 수 있어 . 굳 !

[8] 원본: 이 영화 진짜 미쳣다!!! 너무 재밌음ㅋㅋㅋㅋ
    결과: 영화 진짜 미쳤다 ! 너무 재밌음

[9] 원본: 배우 연기 굳이 훌륭했음, 스토리는 봣지만...
    결과: 배우 연기 굳이 훌륭했음 , 스토리 봤지만 .

[10] 원본: 이건 ㄱㅊ 영화네, 굿굿!
    결과: 이건 괜찮 영화네 , 굳굳 !

실제 데이터 적용 예시
전처리 전 데이터 크기: 150000
결측치 제거 후: 149995
빈 문자열 제거 후: 149607
중복 제거 후: 144478
전처리 전 데이터 크기: 50000
결측치 제거 후: 49997
빈 문자열 제거 후: 49843
중복 제거 후: 48700

전처리 완료!
Train 데이터: 144478개
Test 데이터: 48700개

전처리 결과 샘플:
         id                                       

# 3. 토큰화 (SentencePiece)

In [9]:
!pip install sentencepiece

## 3.1. SentencePiece 모델 학습

In [10]:
import sentencepiece as spm
import os

# 학습용 corpus 파일 생성
corpus_file = 'naver_reviews.train.temp'

with open(corpus_file, 'w', encoding='utf-8') as f:
    for text in train_processed['document']:
        f.write(text + '\n')

# SentencePiece 학습
vocab_size = 8000
model_prefix = 'naver_spm'

spm.SentencePieceTrainer.Train(
    f'--input={corpus_file} '
    f'--model_prefix={model_prefix} '
    f'--vocab_size={vocab_size} '
    f'--model_type=bpe '
    f'--pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 '
    f'--pad_piece=<PAD> --unk_piece=<UNK> --bos_piece=<BOS> --eos_piece=<EOS>'
)

!ls -l naver_spm*

-rw-r--r-- 1 root root 374858 Nov 13 07:24 naver_spm.model
-rw-r--r-- 1 root root 117611 Nov 13 07:24 naver_spm.vocab


In [11]:
# 학습된 SentencePiece 모델 활용 예시
sp = spm.SentencePieceProcessor(model_file=f'{model_prefix}.model')

sample_text = "이 영화 진짜 미쳣다!!! 너무 재밌음ㅋㅋㅋㅋ"
encoded_ids = sp.encode(preprocess_text(sample_text),  # 샘플 text 전처리
                        out_type=int, add_bos=True, add_eos=True)
decoded_text = sp.decode(encoded_ids)

print("Encoded IDs:", encoded_ids)
print("Decoded Text:", decoded_text)


Encoded IDs: [2, 6, 52, 2756, 6458, 13, 26, 1137, 3]
Decoded Text: 영화 진짜 미쳤다 ! 너무 재밌음


In [12]:
# 최대 문장 길이 자르기
def truncate_corpus(corpus, max_length=130):
    """
    corpus: 전처리된 문장 리스트
    max_length: 문장 최대 글자 수
    """
    truncated_corpus = []
    for sentence in corpus:
        truncated_corpus.append(sentence[:max_length])
    return truncated_corpus

문장 자르기를 토큰화와 패딩 사이에 해야 하는 것을 뒤늦게 알아서,, 시간상 일단 이렇게 했습니다.

## 3.2. 텐서로 변환
아래 함수는 SentencePiece로 토큰화된 데이터를 LSTM 등 모델 입력용 tensor로 만드는 함수.  
<br>

1. 매개변수로 토큰화된 문장의 list를 전달하는 대신 온전한 문장의 list 를 전달합니다.

2. 생성된 vocab 파일을 읽어와 { \<word> : \<idx> } 형태를 가지는 word_index 사전과 { \<idx> : \<word>} 형태를 가지는 index_word 사전을 생성하고 함께 반환합니다.

3. 리턴값인 tensor 토큰화한 후 Encoding된 문장입니다. 바로 학습에 사용할 수 있게 Padding은 당연히 해야겠죠?


```
# max length 지정
import matplotlib.pyplot as plt

lens = [len(sp.EncodeAsIds(sen)) for sen in train_processed['document']]
plt.hist(lens, bins=50)
plt.show()

```
import numpy as np
max_len = int(np.percentile(lens, 95))
print(max_len)

In [13]:
def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus:
        tensor.append(torch.tensor(s.EncodeAsIds(sen), dtype=torch.long))

    with open("./naver_spm.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({word:idx})
        index_word.update({idx:word})

    tensor = pad_sequence(tensor, batch_first=True, padding_value=0)

    return tensor, word_index, index_word

# 4. 데이터 로더

In [22]:
from torch.utils.data import TensorDataset, DataLoader, random_split

def create_dataloaders(train_tensor, train_labels, test_tensor, test_labels,
                       batch_size=32, val_ratio=0.2):
    dataset = TensorDataset(tensor, labels)
    val_size = int(len(dataset) * val_ratio)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    test_dataset = TensorDataset(test_tensor, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [15]:
# 전처리 후 문자열 corpus
corpus = train_processed['document'].tolist()
labels = torch.tensor(train_processed['label'].values, dtype=torch.long)

# 1. 문자열 최대 길이 제한
corpus_truncated = truncate_corpus(corpus, 130)

# 2. SentencePiece 모델 로드
sp = spm.SentencePieceProcessor(model_file='naver_spm.model')

# 3. 토큰화 + 패딩
tensor, word_index, index_word = sp_tokenize(sp, corpus_truncated)

# 4. DataLoader 생성
#train_loader, val_loader = create_dataloaders(tensor, labels)

# 확인
#print(f"Train batches: {len(train_loader)}, Validation batches: {len(val_loader)}")
#for batch in train_loader:
#    x, y = batch
#    print("입력 텐서 shape:", x.shape)
#    print("레이블 shape:", y.shape)
#    break

Train batches: 3612, Validation batches: 903
입력 텐서 shape: torch.Size([32, 111])
레이블 shape: torch.Size([32])


In [24]:
# 전처리 후 문자열 corpus
corpus_test = test_processed['document'].tolist()
labels_test = torch.tensor(test_processed['label'].values, dtype=torch.long)

# 1. 문자열 최대 길이 제한
corpus_truncated_test = truncate_corpus(corpus_test, 130)

# 2. SentencePiece 모델 로드
sp = spm.SentencePieceProcessor(model_file='naver_spm.model')

# 3. 토큰화 + 패딩
tensor_test, word_index_test, index_word_test = sp_tokenize(sp, corpus_truncated_test)

TypeError: create_dataloaders() missing 2 required positional arguments: 'test_tensor' and 'test_labels'

In [25]:
train_loader, val_loader, test_loader = create_dataloaders(
    train_tensor=tensor,
    train_labels=labels,
    test_tensor=tensor_test,
    test_labels=labels_test,
    batch_size=32,
    val_ratio=0.2
)

# 5. 모델 정의 및 학습

In [17]:
import torch.nn as nn
import torch.nn.functional as F

# 공통 하이퍼파라미터 정의
VOCAB_SIZE = 8000                  # 단어 사전의 크기
EMBEDDING_DIM = 100                # 임베딩 벡터의 차원
HIDDEN_DIM = 128                   # LSTM의 은닉 상태 차원
OUTPUT_DIM = 1                     # 출력 차원 (긍정=1, 부정=0 -> 1개)
N_LAYERS = 2                       # LSTM 레이어 개수
BIDIRECTIONAL = True               # 양방향 RNN/LSTM 여부
DROPOUT_RATE = 0.2                 # 드롭아웃 비율
PAD_IDX = 0                        # 패딩 인덱스 (0)

In [18]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):
        super().__init__()

        # 1. 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # 2. LSTM 레이어
        self.lstm = nn.LSTM(input_size=embedding_dim,
                           hidden_size=hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           batch_first=True,
                           dropout=dropout)

        # 3. FC 레이어
        fc_input_dim = hidden_dim * 2
        self.fc = nn.Linear(fc_input_dim, output_dim)

        # 4. 드롭아웃
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        embedded = self.embedding(text)

        # 2. LSTM
        _output, (hidden, cell) = self.lstm(embedded)

        # 3. 마지막 레이어의 은닉 상태 결합 (양방향 처리)
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])

        # 4. FC 레이어 통과
        prediction = self.fc(hidden)

        return prediction.squeeze(1)

In [19]:
import torch
import torch.optim as optim
import time

# 0. GPU 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 1. 헬퍼 함수 정의
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# 2. 훈련 함수 정의
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for texts, labels in iterator:
        texts = texts.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels.float())
        acc = binary_accuracy(predictions, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # 클리핑
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# 3. 평가 함수 정의
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for texts, labels in iterator:
            texts = texts.to(device)
            labels = labels.to(device)
            predictions = model(texts)
            loss = criterion(predictions, labels.float())
            acc = binary_accuracy(predictions, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Using device: cuda


In [26]:
# 모델 설정
lstm_model = LSTMModel(
    VOCAB_SIZE,
    EMBEDDING_DIM,
    HIDDEN_DIM,
    OUTPUT_DIM,
    N_LAYERS,
    BIDIRECTIONAL,
    DROPOUT_RATE,
    PAD_IDX
).to(device)

save_path = 'best_model_lstm.pt'
N_EPOCHS = 20
patience = 5
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, lstm_model.parameters()), lr=0.0001)

# Early stopping 변수
best_valid_loss = float('inf')
patience_counter = 0

train_losses, train_accs, valid_losses, valid_accs = [], [], [], []

print(f"\n{'='*60}")
print(f"--- LSTM Model Training starts ---")
print(f"{'='*60}\n")

# 학습 루프
for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(lstm_model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(lstm_model, val_loader, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    # 기록
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    valid_losses.append(valid_loss)
    valid_accs.append(valid_acc)

    # Early Stopping
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(lstm_model.state_dict(), save_path)
        patience_counter = 0
        print(f'\t>> Validation loss improved ({best_valid_loss:.3f}). Model saved.')
    else:
        patience_counter += 1
        print(f'\t>> Validation loss did not improve. Counter: {patience_counter}/{patience}')
        if patience_counter >= patience:
            print(f'--- Early stopping triggered after {epoch+1} epochs ---')
            break

# 테스트 평가
print(f"\n--- Loading best LSTM model for test evaluation ---")
lstm_model.load_state_dict(torch.load(save_path))
test_loss, test_acc = evaluate(lstm_model, test_loader, criterion)

print(f"\n--- LSTM Model Test Results (Best Model) ---")
print(f'\tTest Loss: {test_loss:.3f}')
print(f'\tTest Acc:  {test_acc*100:.2f}%')



--- LSTM Model Training starts ---

Epoch: 01 | Time: 0m 33s
	Train Loss: 0.562 | Train Acc: 69.92%
	 Val. Loss: 0.491 |  Val. Acc: 76.73%
	>> Validation loss improved (0.491). Model saved.
Epoch: 02 | Time: 0m 32s
	Train Loss: 0.443 | Train Acc: 79.44%
	 Val. Loss: 0.429 |  Val. Acc: 80.20%
	>> Validation loss improved (0.429). Model saved.
Epoch: 03 | Time: 0m 32s
	Train Loss: 0.392 | Train Acc: 82.38%
	 Val. Loss: 0.408 |  Val. Acc: 81.58%
	>> Validation loss improved (0.408). Model saved.
Epoch: 04 | Time: 0m 32s
	Train Loss: 0.359 | Train Acc: 84.25%
	 Val. Loss: 0.401 |  Val. Acc: 82.64%
	>> Validation loss improved (0.401). Model saved.
Epoch: 05 | Time: 0m 32s
	Train Loss: 0.335 | Train Acc: 85.67%
	 Val. Loss: 0.388 |  Val. Acc: 82.72%
	>> Validation loss improved (0.388). Model saved.
Epoch: 06 | Time: 0m 32s
	Train Loss: 0.315 | Train Acc: 86.71%
	 Val. Loss: 0.391 |  Val. Acc: 83.21%
	>> Validation loss did not improve. Counter: 1/5
Epoch: 07 | Time: 0m 32s
	Train Loss: 0.

SentencePiece을 사용한 LSTM 모델 학습 결과:  
- Test Loss: 0.383
- Test Acc: 83.23%  